# Openning Data

In [3]:
using SparseArrays
using JuMP
using Ipopt
using GLPK
using LinearAlgebra
using MAT

In [4]:
file = matopen("R5T2.mat")
varnames = names(file)

4-element Array{String,1}:
 "K"
 "L"
 "S"
 "U"

In [5]:
S = read(file, "S")
L = read(file, "L")
U = read(file, "U")
K = read(file, "K")

20

In [6]:
(m,n)=size(S)
(r,c) = size(L)

(13543, 100)

# constructing model

In [7]:
mzeros = zeros(m,1);

In [8]:
function opt_vector0(S,l,u,z)
    model = Model(Ipopt.Optimizer)
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @constraint(model, S*v .== mzeros)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    @NLobjective(model,Min,sum(sqrt(1+z[i]+a[i]) for i=1:r))
    optimize!(model)
    
    return JuMP.value.(v)
end

opt_vector0 (generic function with 1 method)

In [9]:
function opt_vector(S,l,u,z,lambda)
    model = Model(Ipopt.Optimizer)
    
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @variable(model,b[1:m])
    
    # @constraint(model, S*v .== mzeros)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    
    @constraint(model, b .>= S*v)
    @constraint(model, b .>= -S*v)
    
    
    @NLobjective(model,Min,sum(sqrt(1+z[i]+a[i]) for i=1:r) + lambda*sum(b[j] for j=1:m))
    optimize!(model)
    
    return JuMP.value.(v)
end

opt_vector (generic function with 1 method)

In [10]:
function zero_rows(V)
    indexes= zeros(Int64,size(V)[1],1)
    k=1
    for i in 1:size(V)[1]
        if sum(V[i,:].*V[i,:])<1e-5
            indexes[k]= i
            k+=1
        end
    end
    return length(indexes[1:k-1])
end

zero_rows (generic function with 1 method)

In [11]:
function row_squares(V)
    r,c = size(V,1), size(V,2)
    squares = zeros(r,1)
    for i in 1:r
        z=0
        for j in 1:c
           z += (V[i,j])^2
        end
        #squares[i]=sqrt(z)
        squares[i]=z
    end
    return squares
end

row_squares (generic function with 1 method)

In [12]:
function row_squares(V,z)
    r,c = size(V,1), size(V,2)
    for i in 1:r
        z[i]=z[i]+V[i,c]^2
    end
    return z
end

row_squares (generic function with 2 methods)

In [27]:
# we tried lambda = 50,100,120,150 and this was the best result among these choices
lambda = 100

100

In [28]:
l = L[:,1]
u = U[:,1]
z = ones(r,1)
v = opt_vector(S,l,u,z,lambda)
zero_rows(v)

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    58908
Number of nonzeros in Lagrangian Hessian.............:    13543

Total number of variables............................:    23307
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1365
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    43884
        inequality constraints with only lower bounds:    43884
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

13422

In [29]:
V=v;

In [30]:
z = zeros(r,1)
for i in 2:c
    if isdefined(Main, :IJulia)
        Main.IJulia.stdio_bytes[] = 0
    end
    l = L[:,i]
    u = U[:,i]
    z = row_squares(V,z)
    v = opt_vector(S,l,u,z,lambda)
    V = [V v]
    print("**i : ",i,"\n")
    print("***",zero_rows(V),"\n") 
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    58398
Number of nonzeros in Lagrangian Hessian.............:    13543

Total number of variables............................:    23255
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1313
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    43884
        inequality constraints with only lower bounds:    43884
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    58188
Number of nonzeros in Lagrangian Hessian.............:    13543

Total number of variables............................:    23236
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1294
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    43884
        inequality constraints with only lower bounds:    43884
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    57788
Number of nonzeros in Lagrangian Hessian.............:    13543

Total number of variables............................:    23218
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1276
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    43884
        inequality constraints with only lower bounds:    43884
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    57882
Number of nonzeros in Lagrangian Hessian.............:    13543

Total number of variables............................:    23243
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1301
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    43884
        inequality constraints with only lower bounds:    43884
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    58016
Number of nonzeros in Lagrangian Hessian.............:    13543

Total number of variables............................:    23232
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1290
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    43884
        inequality constraints with only lower bounds:    43884
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

...

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    58630
Number of nonzeros in Lagrangian Hessian.............:    13543

Total number of variables............................:    23256
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1314
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    43884
        inequality constraints with only lower bounds:    43884
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    58262
Number of nonzeros in Lagrangian Hessian.............:    13543

Total number of variables............................:    23254
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1312
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    43884
        inequality constraints with only lower bounds:    43884
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    58966
Number of nonzeros in Lagrangian Hessian.............:    13543

Total number of variables............................:    23318
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1376
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    43884
        inequality constraints with only lower bounds:    43884
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    58072
Number of nonzeros in Lagrangian Hessian.............:    13543

Total number of variables............................:    23271
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1329
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    43884
        inequality constraints with only lower bounds:    43884
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    57688
Number of nonzeros in Lagrangian Hessian.............:    13543

Total number of variables............................:    23222
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1280
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    43884
        inequality constraints with only lower bounds:    43884
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [31]:
V

13543×100 Array{Float64,2}:
  0.0           0.0          0.0   0.0          …  0.0           0.0
  0.0           0.0          0.0   0.0             0.0           0.0
  0.0           0.0          0.0   0.0             0.0           0.0
  0.0           0.0          0.0   0.0             0.0           0.0
  0.0           0.0          0.0   0.0             0.0           0.0
  0.0           0.0          0.0   0.0          …  2.64395e-13   0.0
  0.0           0.0          0.0   0.0             0.0           0.0
  0.0           0.0          0.0   0.0             0.0           0.0
  0.0           0.0          0.0   0.0             0.0           0.0
 -6.27913e-14   0.0          0.0   0.0             0.0           0.0
  6.27913e-14   0.0          0.0   0.0          …  0.0           0.0
  0.0           0.0          0.0   0.0             0.0           0.0
  0.0          -1.54408e-17  0.0   0.0             0.0          -2.9339e-19
  ⋮                                             ⋱               
  0

In [32]:
n1 = zero_rows(V)
n2 = zero_rows((S*V)')
n1 ,n2

(12824, 100)

# writing to output file

In [26]:
using DelimitedFiles
open("output.txt","w") do io
    writedlm(io, V,",")
end